In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%store -r student_courses
%store -r student_majors
%store -r stem_paths
%store -r maker_passes
%store -r jacobs_ethnicities
%store -r bcdi
%store -r waivers

In [18]:
# ppsks of everyone who (in data) uses Jacobs for something
ppsks = maker_passes['ppsk']
ppsks = ppsks.append(jacobs_ethnicities['ppsk']).unique()

# all Jacobs students' academic career, either with DESINV course or MakerPass
all_jacobs = student_courses[(student_courses['ppsk'].isin(ppsks)) & (student_courses['undergrad.status'] == 'Undergraduate')]

In [4]:
jacobs_ethnicities[jacobs_ethnicities['year.name'] == '2016 Fall']['course.number'].value_counts()

198    143
15      61
10      43
190     39
22      32
21      28
Name: course.number, dtype: int64

In [17]:
all_jacobs

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,instructor.name,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours
93,2007 Summer,31341,Graduate,Near Eastern Studies,Persian,20,Primary,1,70310,70310.0,Inten Inter Persian,Jaleh Pirnazar,A,Letter Grade,A+,4.0,10.0
94,2007 Summer,33827,Graduate,German,German,10,Primary,1,48655,48655.0,Elem German Wrkshp,Meredith Kolar; Nikolaus Euba,A,Letter Grade,A+,4.0,10.0
95,2007 Summer,169335,Graduate,Near Eastern Studies,Near Eastern Studies,290A,Independent Study,3,66140,66140.0,Special Studies,Hatem Bazian,A,Letter Grade,A+,4.0,4.0
96,2007 Summer,194536,Graduate,Ethnic Studies,Ethnic Studies,135AC,Primary,2,45245,45245.0,U.S. Immigration,Catherine Choy,A,Letter Grade,A+,4.0,4.0
97,2007 Summer,198873,Undergraduate,Near Eastern Studies,Hebrew,N106,Primary,1,50010,50010.0,Elem Bibl Hebrew,Chava Boyarin,A,Letter Grade,A+,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23102,2019 Summer,4271436,Undergraduate,Nutritional Sciences & Tox,Nutritional Science & Tox,10,Primary,2,14561,14561.0,Intro Human Nutr,NaN,B,Letter Grade,B-,2.7,3.0
23103,2019 Summer,4271494,Undergraduate,Psychology,Psychology,166AC,Primary,1,14105,14105.0,Cultural Psychology,NaN,A,Letter Grade,A+,4.0,3.0
23104,2019 Summer,4271527,Undergraduate,Theater Dance & Perf Stds,Theater Dance & Perf Stds,10,Primary,1,14043,14043.0,Fund. Of Acting I,NaN,A,Letter Grade,A,4.0,3.0
23105,2019 Summer,4271531,Undergraduate,Interdisc Social Science Pgms,Global Studies,154M,Primary,3,15353,15353.0,Topics Middle East,NaN,A,Letter Grade,A,4.0,4.0


In [14]:
all_jacobs = all_jacobs.groupby(['ppsk','year.name']).filter(lambda x: (x['course.subject'].isin(['Design Innovation']).any())) 
#single_students = all_jacobs.groupby('ppsk')

KeyboardInterrupt: 

In [8]:
#get ppsks of students from jacobs and find other classes they take
# how many units students take
#all_jacobs = student_courses
all_jacobs

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,instructor.name,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours
93,2007 Summer,31341,Graduate,Near Eastern Studies,Persian,20,Primary,1,70310,70310.0,Inten Inter Persian,Jaleh Pirnazar,A,Letter Grade,A+,4.0,10.0
94,2007 Summer,33827,Graduate,German,German,10,Primary,1,48655,48655.0,Elem German Wrkshp,Meredith Kolar; Nikolaus Euba,A,Letter Grade,A+,4.0,10.0
95,2007 Summer,169335,Graduate,Near Eastern Studies,Near Eastern Studies,290A,Independent Study,3,66140,66140.0,Special Studies,Hatem Bazian,A,Letter Grade,A+,4.0,4.0
96,2007 Summer,194536,Graduate,Ethnic Studies,Ethnic Studies,135AC,Primary,2,45245,45245.0,U.S. Immigration,Catherine Choy,A,Letter Grade,A+,4.0,4.0
97,2007 Summer,198873,Undergraduate,Near Eastern Studies,Hebrew,N106,Primary,1,50010,50010.0,Elem Bibl Hebrew,Chava Boyarin,A,Letter Grade,A+,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23102,2019 Summer,4271436,Undergraduate,Nutritional Sciences & Tox,Nutritional Science & Tox,10,Primary,2,14561,14561.0,Intro Human Nutr,NaN,B,Letter Grade,B-,2.7,3.0
23103,2019 Summer,4271494,Undergraduate,Psychology,Psychology,166AC,Primary,1,14105,14105.0,Cultural Psychology,NaN,A,Letter Grade,A+,4.0,3.0
23104,2019 Summer,4271527,Undergraduate,Theater Dance & Perf Stds,Theater Dance & Perf Stds,10,Primary,1,14043,14043.0,Fund. Of Acting I,NaN,A,Letter Grade,A,4.0,3.0
23105,2019 Summer,4271531,Undergraduate,Interdisc Social Science Pgms,Global Studies,154M,Primary,3,15353,15353.0,Topics Middle East,NaN,A,Letter Grade,A,4.0,4.0
